# Counting Rubik's Snake shapes with restricted rotations

Rubik's Snake has 4 possible rotations, encoded in formula by numbers 0,1,2,3. Number of valid shapes of $n$ wedges is approximately $\Omega(3.64^{n-1})$.

What if we allow only some of these rotations?

Define $\text{SR}_r(n)$ - number of n-wedge snakes when only rotations from set r are allowed.

Then:
* $\text{SR}_{0123}(n) = S(n)$ - unrestricted number of shapes.
* $\text{SR}_{0}(n) = \text{SR}_1(n)=\text{SR}_2(n)=1$ for any n.
* $\text{SR}_{2}(n) = 1$ for n=1,2,3,4 and $\text{SR}_{2}(n)=0$ otherwise.
* For 2 allowed rotations, accounting for symmetries, we have 4 distinct sequences:
   * $\text{SR}_{01}(n)=\text{SR}_{03}(n)$
   * $\text{SR}_{02}(n)$
   * $\text{SR}_{12}(n)=\text{SR}_{23}(n)$
   * $\text{SR}_{13}(n)$
* For 3 allowed rotations, accounting for symmetries, we have 3 distinct sequences:
   * $\text{SR}_{013}(n)$
   * $\text{SR}_{023}(n)=\text{SR}_{012}(n)$
   * $\text{SR}_{123}(n)$
   
Code below computes first terms for these seven interesting sequences. It also estimates asymptotic for these sequence in a form $A \cdot B^{n-1}$.

In [1]:
import numpy as np
import numba
import functools
import time

from rubiks_snake import _push_next_wedge_if_can, _count_shapes_rec, _pop_wedge, \
  _prepare_arena, INIT_WEDGE

@functools.cache
def count_shapes_rec(allowed_rots):
  ALLOWED_ROTS=np.array(list(set(map(int, allowed_rots))))
  @numba.jit("(i8[:],i8[:],i8[:])")
  def f_rec(wedges, cubes, total_count):
    total_count[wedges[0]] += 1
    if wedges[0] == 1: return
    for rot in ALLOWED_ROTS:
      if _push_next_wedge_if_can(rot, wedges, cubes):
        f_rec(wedges, cubes, total_count)
        _pop_wedge(wedges, cubes)
  return f_rec

def count_shapes(n, allowed_rots="0123"):
  wedges, cubes = _prepare_arena(n, INIT_WEDGE)
  total_count = np.zeros(n + 1, dtype=np.int64)
  count_shapes_rec(allowed_rots)(wedges, cubes, total_count)
  return total_count[1:][::-1]

In [2]:
for allowed_rots, n in [("01", 32), ("02", 32), ("12", 32), ("13", 32),
                        ("013", 18), ("023", 20), ("123", 22)]:
  t0 = time.time()
  counts = count_shapes(n, allowed_rots=allowed_rots)
  appr_exp = counts[-1] / counts[-2]
  appr_const = counts[-1] / appr_exp**(n-1)
  print("SR_%s ~ %.03f * %.03f^(n-1)" % (allowed_rots, appr_const, appr_exp))
  print("First %d terms: %s" % (n, ",".join(map(str,counts))))
  print("Time: %.01fs" % (time.time()-t0))
  print("")

SR_01 ~ 1.101 * 1.993^(n-1)
First 32 terms: 1,2,4,8,16,32,64,128,256,512,1024,2048,4096,8192,16384,32768,65520,131016,261944,523728,1046890,2092776,4182704,8360094,16706223,33386122,66706840,133289212,266266438,531752374,1060898222,2114153979
Time: 53.4s

SR_02 ~ 1.757 * 1.754^(n-1)
First 32 terms: 1,2,4,8,13,24,44,81,139,250,450,809,1403,2498,4447,7910,13769,24363,43106,76236,132865,234171,412731,727253,1267901,2228666,3917654,6885484,12004150,21059478,36947904,64816418
Time: 2.8s

SR_12 ~ 1.816 * 1.617^(n-1)
First 32 terms: 1,2,4,8,12,20,32,52,84,136,220,356,576,932,1508,2440,3948,6388,10336,16724,27058,43776,70808,114544,185265,299688,484698,783946,1267770,2050320,3315059,5360440
Time: 0.9s

SR_13 ~ 1.386 * 1.899^(n-1)
First 32 terms: 1,2,4,8,16,32,58,112,216,416,802,1546,2904,5552,10616,20294,38802,74176,140104,266876,508396,968444,1844880,3514190,6647842,12637552,24025376,45674758,86834448,165078850,312425728,593173256
Time: 18.4s

SR_013 ~ 1.149 * 2.937^(n-1)
First 18 terms: 1,3,

Interesting observations:
* $\text{SR}_{01}(n)$ is very close to $2^{n-1}$, and first discrepancy is at 17-th term (65520 instead of 65536).
* $\text{SR}_{02}(n)$ is the number of "flat" shapes, such that if you place it on a table, all